## Coordinate Descent in OLS

- In gradient descent, we update every $\beta$ value in OLS at the same time using the update function (see Gradient Descent section for more details)
$$\begin{aligned}
    \hat{\beta_{t+1}} &= \hat{\beta_{t}} - \eta \frac{\partial L}{\partial \hat{\beta}}
\end{aligned}$$

- This doesn't always work. Some cases where it may fail:
    - In many cases, your descent could become stuck at local minima or saddle points
    - perhaps due to vanishing or exploding gradients, the updating of the coefficients may not converge
    - In high dimensional spaces, the gradient magnitude may be vastly different across dimensions, which can lead to non-convergence or slow convergence

- In such cases, instead of trying to update all coefficients jointly, it is possible to simply update the coefficients one at a time. This is **coordinate descent**

### Theoretical Foundation

- The idea of coordinate descent is extremely similar to gradient descent, except for an additional loop that goes over the coordinates one by one

- First, let's define the loss function

$$\begin{aligned}
    L(\beta) &= \sum_i (y_i - \hat{y_i})^2 \\
    &= \sum_i (y_i - X_i \hat{\beta})^2 
\end{aligned}$$

- The "descent" idea is the same. At each point, we want to find the rate of change of the loss $L$ with respect to a change in $\beta$. Except in this case, we want to do this one $\beta_i$ at a time. To facilitate this, let's rewrite $L(\beta)$ to make things clearer. This formulation is exactly the same as the one above; we simply separate out the parameter $\beta_{k}$ to make it clear that this is the value we are updating

$$\begin{aligned}
    L(\beta) &= \sum_i (y_i - X_i \hat{\beta})^2 \\
    &= \sum_i (y_i - \sum_{j \neq k} (X_{i,j} \hat{\beta_{j}}) - X_{i,k} \hat{\beta_{k}})^2
\end{aligned}$$

- Since we are treating all coefficients besides $k$ as fixed, the first 2 terms in the summation becomes constant! Thus, we define the residual as contributed by the fixed terms as the **partial residual of k**. That is, it tells us what the residual of observation $i$ should be, if we exclude the contribution of $k$

$$\begin{aligned}
    r_i^{(k)} &= y_i - \sum_{j \neq k} (X_{i,j} \hat{\beta_{j}})
\end{aligned}$$

- Written this way, the true residual becomes

$$\begin{aligned}
    r_i &= (y_i - \hat{y_i})^2 \\
    &= y_i - \sum_{j \neq k} (X_{i,j} \hat{\beta_{j}}) - X_{i,k} \hat{\beta_{k}} \\
    &= r_i^{(k)} - X_{i,k} \hat{\beta_{k}}
\end{aligned}$$

- Then the original loss function (which we want to minimize) can be rewritten in terms of the partial residual term. 
$$\begin{aligned}
    \min_{\beta} L(\beta) &= \min_{\beta} \sum_i (y_i - \hat{y_i})^2 \\
    &= \min_{\beta} \sum_i (r_i^{(k)} - X_{i,k} \hat{\beta_{k}})^2
\end{aligned}$$

- Then, computing $\frac{\partial L}{\partial \hat{\beta_{k}}}$:

$$\begin{aligned}
    \frac{\partial L}{\partial \hat{\beta_{k}}} &= \frac{\partial}{\partial \hat{\beta_{k}}} \sum_i (r_i^{(k)} - X_{i,k} \hat{\beta_{k}})^2 \\
    &= \sum_i 2 X_{i,k} \cdot (r_i^{(k)} - X_{i,k} \hat{\beta_{k}}) \\
    &= 0 \\ \\

    \therefore \sum_i 2 X_{i,k} \cdot (r_i^{(k)} - X_{i,k} \hat{\beta_{k}}) &= 0 \\
    \sum_i X_{i,k} \cdot (r_i^{(k)} - X_{i,k} \hat{\beta_{k}}) &= 0 \\
    \sum_i (X_{i,k} r_i^{(k)} - X_{i,k}^2 \hat{\beta_{k}}) &= 0 \\
    \sum_i X_{i,k} r_i^{(k)} &= \sum_i X_{i,k}^2 \hat{\beta_{k}} \\
    \hat{\beta_{k}} &= \frac{\sum_i X_{i,k} r_i^{(k)}}{\sum_i X_{i,k}^2} 

\end{aligned}$$


### Implementation

In [17]:
import numpy as np

BETA = np.array([0.5,5,20]).reshape(-1, 1)
X = np.concatenate(
    (
        np.ones((300,1)), 
        np.random.uniform(0,1,600).reshape(300,-1)
    ),
    axis=1
)
y = X @ BETA + np.random.normal(0, 1, 300).reshape(-1, 1)
print(X.shape, y.shape)

from sklearn.linear_model import LinearRegression
lr = LinearRegression(fit_intercept=False)
lr.fit(X, y)
lr.coef_ ## Able to find "true" coefficients

(300, 3) (300, 1)


array([[ 0.55683854,  5.0426451 , 19.99019941]])

In [73]:
class OLSWithCoordinateDescent:
    def __init__(self):
        self.iter = 100
        self.eta = 1e-2
    
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.coefs_ = np.ones((n_features, 1))

        for _ in range(self.iter):
            for _coef_index, _coef in enumerate(self.coefs_):
                partial_residual_k = (
                    y - #n x 1
                    X @ self.coefs_ + #n x m @ m x 1
                    (self.coefs_[_coef_index] * X[:, _coef_index]).reshape(-1,1) #scalar * n x 1
                )

                self.coefs_[_coef_index] = (X[:, _coef_index].T @ partial_residual_k) / (np.sum(X[:, _coef_index] ** 2))
                
                # print(self.coefs_)
        
    def predict(self, X):
        return X @ self.coefs_
    
ols = OLSWithCoordinateDescent()
ols.fit(X,y)
ols.coefs_

array([[ 0.55683854],
       [ 5.0426451 ],
       [19.99019941]])